In [1]:
!pip install -q lxml

In [11]:
import bs4
import lxml
import pandas
import urllib
from tqdm.auto import tqdm

from urllib import request

In [12]:
""" Scrapping du site insee.fr """

url = 'https://www.insee.fr/fr/statistiques/6683015?sommaire=6683037'
raw_text = request.urlopen(url).read()
page = bs4.BeautifulSoup(raw_text) #,'lxml')
tableau = page.find('table', {'id' : 'produit-tableau-figure2'})
rows = tableau.find_all('tr')

In [21]:
""" Création d'un dictionnaire pour manipuler plus aisément les données """

data_pop = {
    rows[i].text.split("\n")[1] : [rows[i].text.split("\n")[2], rows[1].text.split("\n")[3].replace('\xa0',' ')] for i in range(1,len(rows))
}

In [19]:
""" Conversion en DataFrame pour exporter """

df_pop = pandas.DataFrame(data_pop)

In [20]:
""" Exportation au format json """

df_pop.to_json('data_pop.json')